In [0]:
%pip install polars
%pip install deltalake

In [0]:
%restart_python

In [0]:
import polars as pl
from polars import DataFrame
from polars.catalog.unity.client import Catalog

# UC client initialization
workspace_url = "https://databricks-domain.cloud.databricks.com"
bearer_token = "" # Replace your bearer token
catalog_client = Catalog(workspace_url,bearer_token=bearer_token)

# Parameters definition
catalog_name = "gold_blt"
namespace = "domain1"


# Specify the S3 region in storage_options
storage_options = {
    "AWS_REGION": "eu-west-1"  # Replace with your S3 bucket region
}

# Creation of a managed table

In [0]:
# Table definition
table_name = "pl_jupyter_managed_table"
table_type = "MANAGED"  # Table type (ie, MANAGED or EXTERNAL)
data_source_format = "DELTA"  # Data storage format (ie, PARQUET)

# Schema definition
schema = pl.Schema(
    [("id", pl.Int64), ("name", pl.Utf8)]
)
# Table creation in Unity Catalog
catalog_client.create_table(
    catalog_name=catalog_name,
    namespace=namespace,
    table_name=table_name,
    schema=schema,
    table_type=table_type,
    data_source_format=data_source_format
)

# Creation of a simple dataframe
data = {
    "id": [1, 2, 3],
    "name": ["Alice", "Bob", "Charlie"],
}
df = DataFrame(data)

# Write dataframe in the previously created table
catalog_client.write_table(
    df=df,
    catalog_name=catalog_name,
    namespace=namespace,
    table_name=table_name,
    delta_mode="overwrite",  # Write mode (ie, overwrite)
    storage_options=storage_options
)

## Select managed table

In [0]:
catalog_client.scan_table(catalog_name=catalog_name, namespace=namespace, table_name=table_name).collect()

# Creation of an external table

In [0]:
# Table definition
table_name = "pl_jupyter_external_table"
storage_root = "s3://ome-path/dummy/domain1/polars_jupyter_external"
table_type = "EXTERNAL"  # Table type (ie, MANAGED or EXTERNAL)
data_source_format = "DELTA"  # Data storage format (ie, PARQUET)

# Schema definition
schema = pl.Schema(
    [("id", pl.Int64), ("name", pl.Utf8)]
)

# Table creation in Unity Catalog
catalog_client.create_table(
    catalog_name=catalog_name,
    namespace=namespace,
    table_name=table_name,
    schema=schema,
    table_type=table_type,
    data_source_format=data_source_format,
    storage_root=storage_root
)

# Creation of a simple dataframe
data = {
    "id": [1, 2, 3],
    "name": ["Massil", "Martin", "Pierre"],
}
df = DataFrame(data)

# Write dataframe in the previously created table
catalog_client.write_table(
    df=df,
    catalog_name=catalog_name,
    namespace=namespace,
    table_name=table_name,
    delta_mode="overwrite",  # Write mode (ie, overwrite)
    storage_options=storage_options
)

## Select external table

In [0]:
catalog_client.scan_table(catalog_name=catalog_name, namespace=namespace, table_name=table_name).collect()

# Table deletion

In [0]:
table_to_delete = "pl_jupyter_managed_table"
#table_to_delete = "pl_jupyter_external_table"

#Delete table
catalog_client.delete_table(
    catalog_name=catalog_name,
    namespace=namespace,
    table_name=table_to_delete
)